# Business Intelligence Analyst Workbook

Answer the Following Questions in the Jupyter Notebook
Please do not use any Python packages to solve these problems. (Packages to import data and run SQL is fine).
If you have any questions, reach out to zach.fifelski@prizepicks.com
You can print your results or just write the SQL query if you're having issues with the notebook.
If you're not familiar with Postgres you are more than welcome to write in your prefered syntax

In [1]:
## Keith Wilcox

Here is the link to my dashboards, they are in the zip file too.

https://public.tableau.com/views/SA_Tableau_Workbook/RecencyStatusReactivationPotential?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link

In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
dbname = ""
user = ""
password = ""
host = ""
port = ""


conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')


sa_entry_data = pd.read_csv('SA_entry_data.csv')
sa_signup_data = pd.read_csv('SA_signup_data.csv')

sa_entry_data['entry_date'] = pd.to_datetime(sa_entry_data['entry_date'], format='%m/%d/%Y')
sa_signup_data['reg_date'] = pd.to_datetime(sa_signup_data['reg_date'], format='%m/%d/%Y')
                             

sa_entry_data.to_sql('sa_entry_data', engine, if_exists='replace', index=False)
sa_signup_data.to_sql('sa_signup_data', engine, if_exists='replace', index=False)

298

## 1.

Treat SA_entry_data.csv and SA_signup_data.csv as two tables in a SQL DB named entry_data and signup_data respectively. Write sql queries to answer the following 4 questions

a) What is the total entry amount for all members who signed up in September, 2020?

In [130]:
sql = """
SELECT SUM(e.entry_amount) AS total_entry_amount
FROM sa_entry_data AS e
JOIN sa_signup_data AS s ON e.user_id = s.user_id
WHERE s.reg_date >= '2020-09-01' AND s.reg_date < '2020-10-01'
"""
q1_df = pd.read_sql_query(sql, conn)
q1_df

c:\Users\fishc\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,total_entry_amount
0,1495076.5


b) What is the average entry amount per member for each day of the week (Monday-Sunday) in the month of September, 2020?

In [128]:
sql = """
WITH sum_of_entries_by_user AS (
    SELECT
        entry_date,
        EXTRACT(DOW FROM entry_date) AS day_of_week,
        user_id,
        SUM(entry_amount) AS total_entry_amount
    FROM
        sa_entry_data
    WHERE
        entry_date >= '2020-09-01' AND entry_date < '2020-10-01'
    GROUP BY
        entry_date,
        day_of_week,
        user_id
),
average_entries_by_weekday AS (
    SELECT
        day_of_week,
        AVG(total_entry_amount) AS average_entry_amount
    FROM
        sum_of_entries_by_user
    GROUP BY
        day_of_week
)
SELECT
    CASE day_of_week
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        WHEN 6 THEN 'Saturday'
    END AS day_of_week,
    average_entry_amount
FROM
    average_entries_by_weekday
"""
q2_df = pd.read_sql_query(sql, conn)
q2_df

,day_of_week,average_entry_amount
0,Wednesday,450.233305
1,Monday,453.902863
2,Thursday,455.416780
3,Saturday,438.284150
4,Tuesday,462.457301
5,Sunday,496.520202
6,Friday,456.723744


c) Create an ordered list of the top 5 most popular players (by total entry amount) and their league in the month of September 2020

In [129]:
sql = """
SELECT player, SUM(entry_amount) as total_entry_amount, league from sa_entry_data
WHERE entry_date >= '2020-09-01' AND entry_date < '2020-10-01'
GROUP BY player, league
ORDER BY total_entry_amount DESC
LIMIT 5;
"""
q3_df = pd.read_sql_query(sql, conn)
q3_df

c:\Users\fishc\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,player,total_entry_amount,league
0,Jamal Murray,165348.0,NBA
1,Nikola Jokic,163722.0,NBA
2,LeBron James,162286.5,NBA
3,Jayson Tatum,159838.5,NBA
4,Jimmy Butler,151018.0,NBA


d) Determine each user's contribution (by percentage) to the total entry amount for each player.

The final output should include 3 columns: player_name, user_id, and share. 

Note: You do not need to format the final column with a percentage sign

Example Output:

| player_name | user_id | share |
| --- | --- | --- |
| Lebron James | Fedor | 3.500000 |
| Lebron James | Rachel | 96.500000 |
| Michael Jordan | Fedor | 1.404454 |
| Michael Jordan | Rachel | 98.595546 |

In [120]:
sql = """
WITH sum_of_entries_on_player AS (
    SELECT
        player,
        SUM(entry_amount) AS total_entry_amount
    FROM
        sa_entry_data
    GROUP BY
        player
),
sum_of_entries_by_user AS (
    SELECT
        player,
        user_id,
        SUM(entry_amount) AS user_entry_amount
    FROM
        sa_entry_data
    GROUP BY
        player,
        user_id
)
SELECT
    u.player as player_name,
    u.user_id,
    (u.user_entry_amount / p.total_entry_amount) * 100 AS share
FROM
    sum_of_entries_by_user AS u
JOIN
    sum_of_entries_on_player AS p
ON
    u.player = p.player
ORDER BY
    u.player,
    share DESC;
"""
q4_df = pd.read_sql_query(sql, conn)
q4_df.head()

c:\Users\fishc\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,player_name,user_id,share
0,Aaron Donald,bde8f28f-881b-4c42-b284-9f0ac6d6f09c,9.100857
1,Aaron Donald,419c2f99-3e8c-49b3-9eff-8805f04ac656,8.891118
2,Aaron Donald,8537b3dc-4e1e-421f-92ba-f745ce06a333,8.599307
3,Aaron Donald,e0cab10c-d789-405c-b346-81b9e9ed338c,7.349991
4,Aaron Donald,e2ad8e00-f22d-4dc7-a1a8-740692d2d5d4,7.286157


# This is the end of the SQL/Python portion